# Create time series of snow cover over an area of interest using NDSI thresholding

__Requirements:__
- Area of Interest (AOI) geospatial file: shapefile, geopackage, or other file readable by geopandas
- Google Earth Engine account: sign up for a free account [here](https://code.earthengine.google.com/register).


In [1]:
import ee
import sys
import geopandas as gpd
import matplotlib.pyplot as plt

## Set up paths in directory

In [ ]:
# Name of study site, used in output file names
site_name = "MCS"
# Path to the ndsi-snow-maps code
code_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/Idaho_snow_maps_NDSI/ndsi-snow-maps'
# Full path to area of interest
aoi_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/Idaho_snow_maps_NDSI/MCS/MCSborder_WGS84.shp'
# Path where snow maps will be saved
out_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/Idaho_snow_maps_NDSI/MCS/snow_maps'
# Path where figures will be saved
figures_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/Idaho_snow_maps_NDSI/MCS/snow_maps_figures'

# Authenticate and/or Initialize GEE 
# Change the project according to your GEE account, usually "ee-USERNAME" is your default project
try:
    ee.Initialize(project='ee-raineyaberle')
except:
    ee.Authenticate()
    ee.Initialize(project='ee-raineyaberle')

# Add path to functions
sys.path.append(code_path)
import functions as f

## Define search settings

In [3]:
# Define search settings (dates and months are inclusive)
start_date = '2018-05-01'
end_date = '2024-10-10'
start_month = 1
end_month = 12
mask_clouds = True  # whether to mask clouds before mapping snow
aoi_coverage = 70  # minimum coverage of the area of interest

## Load Area of Interest (AOI)

In [ ]:
# Load AOI
aoi = gpd.read_file(aoi_path)
# Estimate optimal UTM CRS
aoi = aoi.to_crs('EPSG:4326') # reproject to WGS84 lat lon
crs_utm = f.convert_wgs_to_utm(lon=aoi.geometry[0].centroid.coords.xy[0][0], lat=aoi.geometry[0].centroid.coords.xy[1][0])
print('Optimal UTM CRS:', crs_utm)
# Reproject to optimal UTM zone
aoi_utm = aoi.to_crs(crs_utm)

# Plot 
aoi_utm.plot()
plt.xlabel('Easting [m]')
plt.ylabel('Northing [m]')
plt.show()

## Query GEE for imagery, classify snow, and save to file

In [ ]:
# Sentinel-2 Surface Reflectance
print('Sentinel-2 SR:\n----------')
f.query_imagery_classify_snow(aoi_utm, "Sentinel-2_SR", start_date, end_date, start_month, end_month,
                              aoi_coverage, out_path, figures_path, site_name, ndsi_threshold=0.4)

# Landsat 8/9
print('\nLandsat:\n----------')
f.query_imagery_classify_snow(aoi_utm, "Landsat", start_date, end_date, start_month, end_month, 
                              aoi_coverage, out_path, figures_path, site_name, ndsi_threshold=0.4)

print('DONE! :)')